# 2. Query Database SQLite

We will using `chinook sample database` from **sqlitetutorial.net**

Database Reference: [https://www.sqlitetutorial.net/sqlite-sample-database/](https://www.sqlitetutorial.net/sqlite-sample-database/)

There are 11 tables in the chinook sample database.

* **`employees`** table stores employees data such as employee id, last name, first name, etc. It also has a field named **`ReportsTo`** to specify who reports to whom.
* **`customers`** table stores customers data.
* **`invoices`** & **`invoice_items`** tables: these two tables store invoice data. <br>The **`invoices`** table stores invoice header data and the **`invoice_items`** table stores the invoice line items data.
* **`artists`** table stores artists data. It is a simple table that contains only the artist id and name.
* **`albums`** table stores data about a list of tracks. Each album belongs to one artist. However, one artist may have multiple albums.
* **`media_types`** table stores media types such as MPEG audio and AAC audio files.
* **`genres`** table stores music types such as rock, jazz, metal, etc.
* **`tracks`** table stores the data of songs. Each track belongs to one album.
* **`playlists`** & **`playlist_track`** tables: playlists table store data about playlists. Each playlist contains a list of tracks. Each track may belong to multiple playlists. The relationship between the **`playlists`** table and **`tracks`** table is many-to-many. The **`playlist_track`** table is used to reflect this relationship.

<br>

<img src="database/chinook_schema.png" width="800">

In [1]:
# Import SQLite, Create connection and create CursorObject
import sqlite3
connection = sqlite3.connect('database/chinook.db')
cursorObj = connection.cursor()

### Show All Tables from chinook Database

In [2]:
cursorObj.execute("""SELECT name FROM sqlite_master 
                     WHERE type='table' AND name NOT LIKE 'sqlite_%';""").fetchall()

[('albums',),
 ('artists',),
 ('customers',),
 ('employees',),
 ('genres',),
 ('invoices',),
 ('invoice_items',),
 ('media_types',),
 ('playlists',),
 ('playlist_track',),
 ('tracks',)]